# Lesson 3: Reflection and Blogpost Writing

## Setup

In [21]:
import os
from dotenv import load_dotenv
config_list = [
  
    {
        'model': 'gpt4',
        'api_key': ''+(os.environ.get('AZURE_OPENAI_KEY'))+'',
        'base_url':  ''+(os.environ.get('AZURE_OPENAI_ENDPOINT'))+'',
        'api_type': 'azure',
        'api_version': '2024-02-01'
    }
]

## The task!

In [22]:
task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''




## Create a writer agent

In [23]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
     llm_config={
        "config_list": config_list,
    }
)

In [6]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [7]:
print(reply)

Title: Unleashing Potential with DeepLearning.AI

Delve into the world of artificial intelligence through DeepLearning.AI. This pioneering online educational platform, founded by AI luminary Dr. Andrew Ng, empowers learners globally. It offers extensive courses from Neural Networks to Deep Learning Specializations, illuminating the intricacies of cutting-edge AI technology. Distinguish yourself in the evolving technological landscape with certifications that hold global recognition. DeepLearning.AI equips you with AI firepower, driving a robust understanding of AI to make impactful contributions in your field. Enter the future with DeepLearning.AI – your roadmap to make strides in AI-powered digital transformation.


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [10]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
     llm_config={
        "config_list": config_list,
    },
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [11]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unleashing Potential with DeepLearning.AI

Delve into the world of artificial intelligence through DeepLearning.AI. This pioneering online educational platform, founded by AI luminary Dr. Andrew Ng, empowers learners globally. It offers extensive courses from Neural Networks to Deep Learning Specializations, illuminating the intricacies of cutting-edge AI technology. Distinguish yourself in the evolving technological landscape with certifications that hold global recognition. DeepLearning.AI equips you with AI firepower, driving a robust understanding of AI to make impactful contributions in your field. Enter the future with DeepLearning.AI – your roadmap to make strides in AI-powered digital transformation.

--------------

## Nested chat

In [12]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
     llm_config={
        "config_list": config_list,
    },
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [13]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config={
        "config_list": config_list,
    },
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [14]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
     llm_config={
        "config_list": config_list,
    },
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [15]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
     llm_config={
        "config_list": config_list,
    },
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [16]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [17]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [18]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unleashing Potential with DeepLearning.AI

Delve into the world of artificial intelligence through DeepLearning.AI. This pioneering online educational platform, founded by AI luminary Dr. Andrew Ng, empowers learners globally. It offers extensive courses from Neural Networks to Deep Learning Specializations, illuminating the intricacies of cutting-edge AI technology. Distinguish yourself in the evolving technological landscape with certifications that hold global recognition. DeepLearning.AI equips you with AI firepower, driving a robust understanding of AI to make impactful contributions in your field. Enter the future with DeepLearning.AI – your roadmap to make strides in AI-powered digital transformation.

--------------

## Get the summary

In [20]:
print(res.summary)

Title: Master AI with DeepLearning.AI: Your Online AI Learning Platform

Boost your AI prowess with DeepLearning.AI, an exceptional online platform that offers an array of AI courses and AI specializations. Launched by globally recognized AI expert Dr. Andrew Ng, this platform provides AI certification programs that enjoy worldwide validation. With comprehensive resources on Neural Networks to Deep Learning, it demystifies complex AI concepts. While DeepLearning.AI provides unparalleled learning opportunities, individual success depends on the learner's dedication and application. Step into the AI revolution on your terms, embarking on a journey with DeepLearning.AI that prioritizes responsible learning and use of AI technology. For more information [Click here](website link).
